A Rule-Based Chatbot for Postnatal Care Support 
NATURAL LANGUAGE PROCESSİNG SEN534
This chatbot was implemented using Google Collab
created by: ASMA BUSHAALA

In [ ]:
#import all libraries
import json
import random
import nltk
import numpy as np
from tensorflow.keras.models import load_model
import pickle
import time
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [ ]:
# Load JSON data
with open('PostnatalCareData.json') as file:
    data = json.load(file)

In [ ]:
# Count number of intents or objects
num_intents = len(data['ourIntents'])

print(f"Number of intent objects: {num_intents}")

In [ ]:
#preprocessing
nltk.download('punkt_tab')
lemmatizer = WordNetLemmatizer()
all_words = []
classes = []
documents = []

for intent in data['ourIntents']:
    for pattern in intent['patterns']:
        tokens = nltk.word_tokenize(pattern)
        all_words.extend(tokens)
        documents.append((tokens, intent['tag']))
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

#lemmatize and lower words
all_words = [lemmatizer.lemmatize(word.lower()) for word in all_words if word.isalpha()]
all_words = sorted(set(all_words))
classes = sorted(set(classes))

In [ ]:
#bag of words
training_data = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(w.lower()) for w in doc[0]]
    for word in all_words:
        bag.append(1 if word in pattern_words else 0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training_data.append([bag, output_row])

In [ ]:
training_data = np.array(training_data, dtype=object)
X = np.array(list(training_data[:, 0]))
y = np.array(list(training_data[:, 1]))

#Train - test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create model
model = Sequential()
model.add(Dense(128, input_shape=(len(X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Train
model.fit(X_train, y_train, epochs=200, batch_size=8, verbose=1)

#save model and data ## i commented this because its additional step this is not required
#model.save('chatbot_model.h5')
#pickle.dump({'words': all_words, 'classes': classes, 'data': data}, open('chatbot_data.pkl', 'wb'))

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
model = load_model('chatbot_model.h5')
data = pickle.load(open('chatbot_data.pkl', 'rb'))

words = data['words']
classes = data['classes']
intents = data['data']['ourIntents']

def clean_input(sentence):
    tokens = nltk.word_tokenize(sentence)
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.isalpha()]
    return tokens

def bag_of_words(sentence, words):
    tokens = clean_input(sentence)
    bag = [0] * len(words)
    for token in tokens:
        for i, w in enumerate(words):
            if w == token:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence, words)
    result = model.predict(np.array([bow]))[0]
    threshold = 0.6 
    if max(result) < threshold:
        return None
    return classes[np.argmax(result)]

def get_response(intent_tag):
    for intent in intents:
        if intent['tag'] == intent_tag:
            return random.choice(intent['responses'])
    return "Sorry, I didn't understand that."

total_inputs = 0
matched_inputs = 0
fallback_count = 0

##chatting loop 
total_inputs = 0
matched_inputs = 0
fallback_count = 0
response_times = []

print("👶 Postnatal Care Chatbot is ready! Type 'exit' to quit.")

while True:
    user_input = input("You: ").strip().lower()

    if user_input in ["quit", "exit", "stop", "bye","ok bye"]:
        print("Bot: Thanks for chatting, mama! Take care 💜")
        break

    total_inputs += 1

    start_time = time.time()

    intent_tag = predict_class(user_input)

    if intent_tag:
        matched_inputs += 1
        response = get_response(intent_tag)
    else:
        fallback_count += 1
        response = ("Sorry, I didn’t understand. "
                    "You can ask about baby sleep, feeding, recovery, or mental health.")

    end_time = time.time()
    elapsed = end_time - start_time
    response_times.append(elapsed)

    print(f"Bot: {response}")
    print(f"(⏱ Response time: {elapsed:.2f} seconds)")

#after chat ends display session summary (evaluation)
if total_inputs > 0:
    intent_match_rate = (matched_inputs / total_inputs) * 100
    fallback_rate = (fallback_count / total_inputs) * 100
    avg_response_time = sum(response_times) / len(response_times)

    print("\n📊 Session Summary:")
    print(f"Total messages: {total_inputs}")
    print(f"Matched intents: {matched_inputs}")
    print(f"Fallbacks: {fallback_count}")
    print(f"Intent Match Rate: {intent_match_rate:.2f}%")
    print(f"Fallback Rate: {fallback_rate:.2f}%")
    print(f"Average Response Time: {avg_response_time:.2f} seconds")
